### GARCH Modeling for Volatility Forecasting (SMALL LoBM)

_In this notebook, we focus on forecasting volatility rather than returns. We use the GARCH(1,1) model, a widely adopted approach for capturing time-varying volatility in financial time series._

**Key Steps:**
 1. Load the dataset for the SMALL LoBM portfolio.
 2. Split the data into training and testing sets (similar date split as before).
 3. Specify and fit a GARCH(1,1) model using the `arch` library.
 4. Generate volatility forecasts for the test period.
 5. Compare predicted volatility to realized volatility (e.g., squared returns) and compute performance metrics such as MAE or RMSE.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from arch import arch_model
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [2]:
plt.style.use('seaborn-v0_8-dark-palette')
sns.set_context("talk")

In [ ]:
file_path = r"\data\Developed_6_Portfolios_ME_BE-ME_cleaned_decimals.csv"

# Load the dataset
df_garch = pd.read_csv(file_path, parse_dates=True, index_col='date')
print("Dataset loaded. Shape:", df_garch.shape)
print(df_garch.head())

Dataset loaded. Shape: (414, 6)
            SMALL LoBM  ME1 BM2  SMALL HiBM  BIG LoBM  ME2 BM2  BIG HiBM
date                                                                    
1990-07-01      0.0243   0.0191      0.0097    0.0064   0.0168    0.0139
1990-08-01     -0.1222  -0.1149     -0.1055   -0.0951  -0.1000   -0.1021
1990-09-01     -0.1082  -0.1001     -0.0938   -0.1177  -0.1079   -0.1160
1990-10-01      0.0520   0.0350      0.0054    0.1304   0.1020    0.0874
1990-11-01     -0.0312  -0.0105     -0.0086   -0.0345  -0.0242   -0.0345


In [5]:
portfolio_name = "SMALL LoBM"
returns = df_garch[portfolio_name].dropna()

In [6]:
print(f"\nUsing {portfolio_name} for GARCH modeling. Data shape:", returns.shape)
print(returns.head())


Using SMALL LoBM for GARCH modeling. Data shape: (414,)
date
1990-07-01    0.0243
1990-08-01   -0.1222
1990-09-01   -0.1082
1990-10-01    0.0520
1990-11-01   -0.0312
Name: SMALL LoBM, dtype: float64


In [7]:
train = returns.loc[:'2015-12-31']
test = returns.loc['2016-01-01':]

print("Training set period:", train.index.min().strftime('%Y-%m'), "to", train.index.max().strftime('%Y-%m'))
print("Testing set period:", test.index.min().strftime('%Y-%m'), "to", test.index.max().strftime('%Y-%m'))


Training set period: 1990-07 to 2015-12
Testing set period: 2016-01 to 2024-12


We Fit a GARCH(1,1) Model on the Training Set

_A GARCH(1,1) model typically suffices for many financial time series, capturing volatility clustering. The `arch` library expects return data (not squared returns), and internally models the variance process._

In [8]:
garch_model = arch_model(train, p=1, q=1, mean='Zero', vol='GARCH', dist='normal')

# Fit the model
garch_fit = garch_model.fit(update_freq=5, disp='off')
print(garch_fit.summary())

                       Zero Mean - GARCH Model Results                        
Dep. Variable:             SMALL LoBM   R-squared:                       0.000
Mean Model:                 Zero Mean   Adj. R-squared:                  0.003
Vol Model:                      GARCH   Log-Likelihood:                474.939
Distribution:                  Normal   AIC:                          -943.879
Method:            Maximum Likelihood   BIC:                          -932.708
                                        No. Observations:                  306
Date:                Tue, Feb 18 2025   Df Residuals:                      306
Time:                        03:27:30   Df Model:                            0
                               Volatility Model                              
                 coef    std err          t      P>|t|       95.0% Conf. Int.
-----------------------------------------------------------------------------
omega      1.5731e-04  1.067e-04      1.475      0.140 

c:\Users\GIORDANO\Desktop\financial-time-series-forecasting\env\Lib\site-packages\arch\univariate\base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.002983. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(


Generating the volatility forecasts 

In [9]:
# Number of steps to forecast = length of the test set
n_test = len(test)

# Forecast the variance over the test horizon
garch_forecast = garch_fit.forecast(horizon=n_test, start=train.index[-1], reindex=False)
forecasted_variance = garch_forecast.variance.iloc[-1].values # setting 1month horizon
pred_variance = pd.Series(forecasted_variance, index=test.index, name='ForecastedVar')

# Convert variance to standard deviation
pred_volatility = np.sqrt(pred_variance)
pred_volatility.name = 'ForecastedVol'

print("First few forecasted variances:")
print(pred_variance.head())
print("\nFirst few forecasted volatilities:")
print(pred_volatility.head())

First few forecasted variances:
date
2016-01-01    0.001690
2016-02-01    0.001756
2016-03-01    0.001818
2016-04-01    0.001878
2016-05-01    0.001934
Name: ForecastedVar, dtype: float64

First few forecasted volatilities:
date
2016-01-01    0.041108
2016-02-01    0.041904
2016-03-01    0.042643
2016-04-01    0.043331
2016-05-01    0.043972
Name: ForecastedVol, dtype: float64


Comparing Forecasted Volatility to Realized Volatility


A common proxy for realized volatility is the squared return (or the absolute return). We'll use squared returns here for consistency with the GARCH variance concept. Then we compute error metrics such as MAE or RMSE between the forecasted standard deviation and the realized standard deviation.


In [10]:
# Realized variance ~ squared returns
realized_variance = test**2
realized_variance.name = 'RealizedVar'
realized_vol = np.sqrt(realized_variance)
realized_vol.name = 'RealizedVol'

# Align forecast and realized series
df_compare = pd.concat([realized_vol, pred_volatility], axis=1).dropna()
print(df_compare.head())

# Compute error metrics (e.g., on volatility, not variance)
mae_vol = mean_absolute_error(df_compare['RealizedVol'], df_compare['ForecastedVol'])
rmse_vol = np.sqrt(mean_squared_error(df_compare['RealizedVol'], df_compare['ForecastedVol']))

print(f"\nVolatility Forecast Error Metrics:\nMAE: {mae_vol:.4f}\nRMSE: {rmse_vol:.4f}")

            RealizedVol  ForecastedVol
date                                  
2016-01-01       0.1053       0.041108
2016-02-01       0.0051       0.041904
2016-03-01       0.0836       0.042643
2016-04-01       0.0261       0.043331
2016-05-01       0.0160       0.043972

Volatility Forecast Error Metrics:
MAE: 0.0323
RMSE: 0.0378


In [14]:
plt.figure(figsize=(12, 6))
plt.plot(df_compare['RealizedVol'], label='Realized Volatility', color='blue')
plt.plot(df_compare['ForecastedVol'], label='Forecasted Volatility', color='red', linestyle='--')
# add confidence intervals
plt.fill_between(df_compare.index, df_compare['ForecastedVol'] - 2*np.sqrt(pred_variance), df_compare['ForecastedVol'] + 2*np.sqrt(pred_variance), color='red', alpha=0.2)
plt.title(f'GARCH(1,1) Volatility Forecast for {portfolio_name}')
plt.xlabel('Date')
plt.ylabel('Volatility')
plt.legend()
plt.savefig('plots/garch_forecast.png')
plt.close()

we will try to achieve major results by using more simulations to forecast volatility 


In [15]:
n_test = len(test)
garch_forecast = garch_fit.forecast(
    horizon=n_test,
    method='simulation',
    simulations=1000,  # adjust as needed
    reindex=False
)

In [18]:
# simulated variances 
simulated_variances = garch_forecast.simulations.values
mean_simulated_variance = np.mean(simulated_variances, axis=1)



In [19]:
# computing confidence intervals by taking percentiles of the simulations.
mean_variance = simulated_variances.mean(axis=1)  
lower_var_ci = np.percentile(simulated_variances, 2.5, axis=1)
upper_var_ci = np.percentile(simulated_variances, 97.5, axis=1)


In [20]:
mean_vol = np.sqrt(mean_variance)
lower_vol_ci = np.sqrt(lower_var_ci)
upper_vol_ci = np.sqrt(upper_var_ci)

C:\Users\GIORDANO\AppData\Local\Temp\ipykernel_1728\3726145924.py:1: RuntimeWarning: invalid value encountered in sqrt
  mean_vol = np.sqrt(mean_variance)
C:\Users\GIORDANO\AppData\Local\Temp\ipykernel_1728\3726145924.py:2: RuntimeWarning: invalid value encountered in sqrt
  lower_vol_ci = np.sqrt(lower_var_ci)


In [22]:
dates_test = test.index  # or create a date range if needed
mean_vol_reshaped = mean_vol.reshape(-1)  # Flatten the array
lower_vol_ci_reshaped = lower_vol_ci.reshape(-1)  # Flatten the array
upper_vol_ci_reshaped = upper_vol_ci.reshape(-1)  # Flatten the array


In [ ]:
realized_vol = np.sqrt(test**2)  # or your chosen realized volatility measure

plt.figure(figsize=(12, 6))
plt.plot(realized_vol, label='Realized Volatility', color='blue')
plt.plot(dates_test, mean_vol_reshaped, label='Forecasted Volatility', color='red', linestyle='--')

# Filter out nan values for confidence intervals
mask = ~np.isnan(upper_vol_ci_reshaped) & ~np.isnan(lower_vol_ci_reshaped)
valid_dates = dates_test[mask]
valid_upper = upper_vol_ci_reshaped[mask]
valid_lower = lower_vol_ci_reshaped[mask]

# Fill the CI band with valid values only
if len(valid_dates) > 0:  
    plt.fill_between(
        valid_dates,
        valid_lower,
        valid_upper,
        color='pink',
        alpha=0.3,
        label='95% Confidence Interval'
    )

plt.title('GARCH(1,1) Volatility Forecast for SMALL LoBM with Confidence Intervals')
plt.xlabel('Date')
plt.ylabel('Volatility')
plt.legend()
plt.savefig('plots/garch_forecast_confidence_intervals.png')
plt.close()